<h1>SidLabs_Segmenting Neighborhoods in Torronto</h1>
<h3>Question#1</h3>
<h4>Using pandas to transform the data in the provided table from Wikipedia page into pandas dataframe.</h4>
<p>1.1 Importing libraries and solving environments<p>

In [12]:
import numpy as np # importing numpy
import pandas as pd # importing pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json # importing JSON
#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert to latitude and longitude
import requests # importing requests
from pandas.io.json import json_normalize # JSON into dataframe
import matplotlib.cm as cm # importing matplotlib
import matplotlib.colors as colors

from sklearn.cluster import KMeans # importing K-Means
from sklearn.datasets.samples_generator import make_blobs
#!conda install -c conda-forge folium=0.5.0 --yes
import folium # importing folium
from bs4 import BeautifulSoup
import lxml
print("all done here!")

all done here!


<p>1.2  Scraping and downloading the data using BeautifulSoup and loading it into Pandas dataframe.</p>

In [13]:
r = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
soup = BeautifulSoup(r.text, 'html.parser')
table=soup.find('table', attrs={'class':'wikitable sortable'})

#heads...
headers=table.findAll('th')
for i, head in enumerate(headers): headers[i]=str(headers[i]).replace("<th>","").replace("</th>","").replace("\n","")
rows=table.findAll('tr')
rows=rows[1:len(rows)]

# skipping symbols and line feeds...
for i, row in enumerate(rows): rows[i] = str(rows[i]).replace("\n</td></tr>","").replace("<tr>\n<td>","")

df=pd.DataFrame(rows)
df[headers] = df[0].str.split("</td>\n<td>", n = 2, expand = True) 
df.drop(columns=[0],inplace=True)

df = df.drop(df[(df.Borough == "Not assigned")].index)
df.Neighbourhood.replace("Not assigned", df.Borough, inplace=True)
df.Neighbourhood.fillna(df.Borough, inplace=True)
df=df.drop_duplicates()

# extract titles...
df.update(
    df.Neighbourhood.loc[
        lambda x: x.str.contains('title')
    ].str.extract('title=\"([^\"]*)',expand=False))

df.update(
    df.Borough.loc[
        lambda x: x.str.contains('title')
    ].str.extract('title=\"([^\"]*)',expand=False))

df.update(
    df.Neighbourhood.loc[
        lambda x: x.str.contains('Toronto')
    ].str.replace(", Toronto",""))
df.update(
    df.Neighbourhood.loc[
        lambda x: x.str.contains('Toronto')
    ].str.replace("\(Toronto\)",""))

# combining neighborhoods with the same post code...
df2 = pd.DataFrame({'Postcode':df.Postcode.unique()})
df2['Borough']=pd.DataFrame(list(set(df['Borough'].loc[df['Postcode'] == x['Postcode']])) for i, x in df2.iterrows())
df2['Neighborhood']=pd.Series(list(set(df['Neighbourhood'].loc[df['Postcode'] == x['Postcode']])) for i, x in df2.iterrows())
df2['Neighborhood']=df2['Neighborhood'].apply(lambda x: ', '.join(x))
df2.dtypes

Postcode        object
Borough         object
Neighborhood    object
dtype: object

<p>1.2.1 having a look at our dataframe</p>

In [15]:
df2.head(10)

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park (Toronto),Queen's Park
5,M9A,Downtown Toronto,Queen's Park
6,M1B,"Scarborough, Toronto","Rouge, Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


<p>1.3 using the .shape method to print the number of rows of the dataframe.</p>

In [11]:
df2.shape

(103, 3)

<h3>Question#2</h3>
<h4>Using the Geocoder package or the csv file to create dataframe with longitude and latitude values.</h4>
<p>2.1 Downloading the toronto longitude and latitude data from cocl.us site and exploring the dataframe.

In [22]:
#!wget -q -O 'Toronto_long_lat_data.csv'  http://cocl.us/Geospatial_data
df_lon_lat = pd.read_csv('Toronto_long_lat_data.csv')
df_lon_lat.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


<p> 2.1.1 Converting the column name 'Postal Code" on df_lon_lat to match "Postcode" from df2.</p>

In [28]:
df_lon_lat.columns=['Postcode','Latitude','Longitude']
df_lon_lat.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


<p> 2.2 merging df2 and df_lon_lat into df3. </p>

In [30]:
df3 = pd.merge(df2,
                 df_lon_lat[['Postcode','Latitude', 'Longitude']],
                 on='Postcode')
df3.head(15)

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park (Toronto),Queen's Park,43.662301,-79.389494
5,M9A,Downtown Toronto,Queen's Park,43.667856,-79.532242
6,M1B,"Scarborough, Toronto","Rouge, Malvern",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


<h3>Question#3</h3>
<p>
   Exploring and clustering the neighborhoods in Toronto. 
   Also use geopy library to get the latitude and longitude values for Toronto.
</p>
<p> 3.1 Importing libraries and solving environments </p>

In [31]:
from geopy.geocoders import Nominatim
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
#!conda install -c conda-forge folium=0.5.0 --yes
import folium
print('All done, Plz proceed...')

All done, Plz proceed...


3.2 Using the geolocator library to obtain the longitude and latitude values for Toronto</p>

In [2]:
address = 'Toronto, ON'
geolocator = Nominatim(user_agent="Toronto")
location = geolocator.geocode(address)
latitude_toronto = location.latitude
longitude_toronto = location.longitude
print('The coordinates for Toronto are {}, {}.'.format(latitude_toronto, longitude_toronto))

The coordinates for Toronto are 43.653963, -79.387207.


<p>3.2 plotting the neighborhoods on the Toronto map to visualize</p>

In [35]:
map_toronto = folium.Map(location=[latitude_toronto, longitude_toronto], zoom_start=10)

# adding markers to map ...
for lat, lng, borough, Neighborhood in zip(df3['Latitude'], df3['Longitude'], df3['Borough'], df3['Neighborhood']):
    label = '{}, {}'.format(Neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='green',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto